In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
from utilities import *
from models import *

import joblib

In [3]:
def create_window(dataset, win_size, start_index=0, end_index=None):
    data = []

    start_index = start_index + win_size
    if end_index is None:
        end_index = len(dataset)

    for i in range(start_index, end_index):
        indices = range(i-win_size, i)
        data.append(dataset[scanners].iloc[indices].values)
    return np.array(data)

In [4]:
ble = read_ble('../Data/ble.csv')

In [5]:
coor = np.array([22, 6])

In [6]:
def impute(data):
    data.update(min_max_scaling(data))
    data.update(data.rolling('1s').mean())
    data.ffill(inplace=True)

def reg_impute(data):
    for s in scanners:
        dx = coor[0] - ble[s]['X']
        dy = coor[1] - ble[s]['Y']
        d = np.sqrt(np.square(dy) + np.square(dx))
        reg = joblib.load('../Models/linear_distance_models/{}.joblib'.format(s))
        ss = pd.Series(reg.predict(d.reshape((-1, 1)))).repeat(data.shape[0])
        ss.index = data.index
        data[s].fillna(ss, inplace=True)

In [7]:
model = tf.keras.models.load_model("../Models/LSTM_basic.h5")

In [8]:
client = InfluxDBClient()

In [9]:
data = client.retrieveData(seconds=3, beacon="0117C55D14E4")

In [10]:
data = format_data(data)

In [12]:
impute(data)
reg_impute(data)

In [13]:
data

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
2020-01-08 02:49:38.064000+00:00,0.275194,0.508436,0.4,0.341812,0.397968,0.473437,0.419770,0.255418,0.404447,0.149574,0.368577,0.495615,0.47342,0.169951,0.520728,0.244721,0.28139
2020-01-08 02:49:38.107000+00:00,0.275194,0.508436,0.4,0.341812,0.397968,0.473437,0.483333,0.255418,0.404447,0.149574,0.368577,0.495615,0.47342,0.169951,0.520728,0.244721,0.28139
2020-01-08 02:49:38.134000+00:00,0.275194,0.508436,0.4,0.341812,0.397968,0.473437,0.483333,0.255418,0.404447,0.149574,0.368577,0.500000,0.47342,0.169951,0.520728,0.244721,0.28139
2020-01-08 02:49:38.304000+00:00,0.275194,0.508436,0.4,0.341812,0.397968,0.473437,0.408333,0.255418,0.404447,0.149574,0.368577,0.500000,0.47342,0.169951,0.520728,0.244721,0.28139
2020-01-08 02:49:38.385000+00:00,0.275194,0.508436,0.4,0.341812,0.397968,0.473437,0.408333,0.255418,0.404447,0.149574,0.368577,0.500000,0.47342,0.169951,0.520728,0.466667,0.28139
2020-01-08 02:49:38.437000+00:00,0.233333,0.508436,0.4,0.341812,0.397968,0.473437,0.408333,0.255418,0.404447,0.149574,0.368577,0.500000,0.47342,0.169951,0.520728,0.466667,0.28139
2020-01-08 02:49:38.518000+00:00,0.233333,0.508436,0.4,0.300000,0.397968,0.473437,0.408333,0.255418,0.404447,0.149574,0.483333,0.500000,0.50000,0.169951,0.520728,0.466667,0.28139
2020-01-08 02:49:38.580000+00:00,0.233333,0.508436,0.4,0.300000,0.397968,0.473437,0.408333,0.255418,0.404447,0.149574,0.483333,0.500000,0.50000,0.169951,0.520728,0.466667,0.28139
2020-01-08 02:49:38.781000+00:00,0.233333,0.508436,0.4,0.300000,0.397968,0.473437,0.408333,0.255418,0.183333,0.149574,0.483333,0.500000,0.50000,0.169951,0.520728,0.466667,0.28139
2020-01-08 02:49:38.901000+00:00,0.233333,0.508436,0.4,0.300000,0.397968,0.473437,0.408333,0.255418,0.183333,0.149574,0.491667,0.500000,0.50000,0.169951,0.520728,0.466667,0.28139


In [14]:
data = create_window(data, 10)

In [15]:
coor = model.predict(data).mean(axis=0)

ValueError: Error when checking input: expected lstm_input to have shape (10, 16) but got array with shape (10, 17)

In [ ]:
np.expand_dims(coor, axis=0)